#### Использованные библиотеки

In [4]:
!pip install -r requirements.txt

In [144]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torchvision.datasets import MNIST, FashionMNIST

import wandb
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader

from pl_base import BestValidCallback
from datagen import SubLoader, TaskAugmentor
from models import GPT
from utils import gpt_params, g_transform

import logging
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)

from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

torch.set_float32_matmul_precision('medium')
device = "cuda" if torch.cuda.is_available() else "cpu"
device

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'cpu'

### Учимся учиться

#### Setup

Одна из самых интересных вещей в этой статье - то, что трансформер, если ему дать достаточно много задач, начинает не просто запоминать задачи, а вычленять общие закономерности, чтобы решать такие задачи, которых он ещё не видел. Мы будем смотреть это через такой же метатест, что и в `3. Metatrain.ipynb`. Вкратце: даём модели последовательность в 99 примеров, смотрим на качество на 100-м. Разнциа лишь в том, что интересует нас теперь лишь трансформер, а также смотреть будем не только на 100-й пример, но и на аккураси в целом по мере обработки последовательности

На картинке ниже показан график улучшения аккураси в зависимости от числа тасок. Суть здесь в том, что аккураси у каждого отдельного элемента последовательности может быть сколь угодно высокий, например `[0.9., 1., 0.9, 1.]`, но значимое улучшение составит лишь 0.1, модель не научилась обрабатывать таску в совокупности, она просто запомнила какое-то оптимальное решение задачи. Если же улучшение наоборот большое, то модель хорошо обобщает и вне зависимости от последовательности выдаёт правильный ответ, потому что научилась учиться. Чем больше тасок, тем больше эффект, фазовый переход происходит примерно на числе в $2^{13}$

<img src="https://cdn.discordapp.com/attachments/674191702906503199/1194420922837372999/image.png?ex=65b04a2b&is=659dd52b&hm=04fc28218f048fa96b685c2df1380f998078f7561b1a8ba18b1b20aeae248022&" width="500px">

Помимо этого посмотрим на сам график улучшения аккураси. В случае трансформера это можно сделать через маску - так, чтобы модель видела лишь часть данных при предсказании, а аккураси будет означать качество на 100-м лейбле. Если качество растёт, хоть как-то, то модель научилась извлекать информацию из последовательности - то есть научилась учиться. В нашем сетапе будем делать это для трансформера на $2^{16}$ тасках, как и в статье

<img src="https://cdn.discordapp.com/attachments/674191702906503199/1194422652597710919/image.png?ex=65b04bc7&is=659dd6c7&hm=596a6d1b05da565f078bfb7dcf3961097b65cc6ad88cc096c66154d4e457d814&" width="500px">

Конфиг, трансформы и датасеты здесь такие же, как в предыдущих экспериментах, как самые удачные. Нам понадобится 3 валидации: MNIST с теми же тасками, MNIST с новыми тасками и FashionMNIST с новыми тасками. Учиться будем либо на MNIST, либо на FashionMNIST

#### Train

In [ ]:
mnist_test = SubLoader(MNIST('./datasets', train=True, transform=g_transform))
fashion_mnist_test = SubLoader(FashionMNIST('./datasets', train=True, transform=g_transform))

Для подсчёта аккураси понадобится кастомная функция. Она генерирует маски до $i$-го токена, делает предсказание модели, имея в виду лишь $i$ токенов, подсчитывает качество для каждого $i$ и его общий прирост - разница в аккураси между обучением на 99 и обучением на 1

In [147]:
def seq_accuracy(loader):
    res = torch.zeros(99)
    for i in range(99):
        attention_mask = torch.zeros(len(loader), 100)
        attention_mask[:, :i] = 1
        pred = F.softmax(
            model(loader.dataset.data, attention_mask=attention_mask), 1
        ).argmax(1)
        res[i] += (pred[:, -1] == loader.dataset.targets[:, -1]).float().mean()
    return res, res[-1] - res[0]

Логику вытаскивания даталоадеров лучше сразу засунуть в функцию. Её лучше вообще не менять. Число тасок в тесте это более изящный способ фиксации сида - оцениваем по каждой, затем усредняем

In [ ]:
def augment_data(
    n_tasks, draw_sequence, batch_size, train_dataset, seed
):
    train_samples = 1
    train_augmentor = TaskAugmentor(n_tasks, draw_sequence=True, random_state=seed)
    seen_augmentor = TaskAugmentor(256, draw_sequence=True, random_state=seed)
    unseen_augmentor = TaskAugmentor(256, draw_sequence=True, random_state=seed+1)
    train_loader = DataLoader(
        train_augmentor.transform(train_dataset, train_samples),
        batch_size=batch_size, shuffle=True
    )
    valid_loader = DataLoader(
        seen_augmentor.transform(mnist_test, 1),
        batch_size=1, shuffle=False
    )
    test_loader = DataLoader(
        unseen_augmentor.transform(mnist_test, 1),
        batch_size=1, shuffle=False
    )
    test_fashion_loader = DataLoader(
        unseen_augmentor.transform(fashion_mnist_test, 1),
        batch_size=1, shuffle=False
    )
    return train_loader, valid_loader, test_loader, test_fashion_loader

Функция ниже запускает обучение и логгирует результаты на wandb, [там](https://wandb.ai/lerostre/gpicl?workspace=user-lerostre) можно на них полюбоваться. Ключевая проблема - модели реально долго учить, они выходят на плато, поэтому EarlyStopping. результаты как обычно в датафрейм. Я запускал только для одного сида, потому что нету времени, а результаты и так видно

Основные параметры - трейн, у нас их 2, число тасок для перебора, таски, на которых замеряем улучшение аккураси, батч сайз для даталоадера, куда сохранять 100-е аккураси, куда сохранять аккураси от 1 до 99, сид, и конфиг для трансформера

In [149]:
def train_for_sequence_improvement(
    train_dataset: SubLoader,
    n_tasks_list: np.ndarray = 2**np.arange(0, 19),
    n_tasks_for_full_check: int = [2**16],
    batch_size: int = 128,
    output_csv: str = "last_acc.csv",
    seq_res_output: str = "mnist_seq_acc",
    random_state: int = 69,
    gpt_params = gpt_params,
    log=True
):
    logs = []
    for n_tasks in n_tasks_list:

        wandb_logger = None
        if log:
            # change if necessary
            wandb.init(project="gpicl", name=f"gpt_2^{int(np.log2(n_tasks))}", tags=["gpt"])
            wandb_logger = WandbLogger(log_model=False)
        # get loaders=[train, "seen_mnist", "unseen_mnist", "unseen_fashion_mnist"]
        loaders = augment_data(
            n_tasks, draw_sequence=True, seed=random_state,
            batch_size=batch_size, train_dataset=train_dataset
        )
        model_name, model = ("transformer", GPT(**gpt_params))

        # training loop, free to adjust
        trainer = pl.Trainer(
            precision="16",
            accelerator="gpu",
            logger=wandb_logger,
            enable_progress_bar=False,
            max_steps=100000,
            enable_model_summary=False,
            callbacks=[EarlyStopping(
                monitor="train_accuracy",
                min_delta=0.025,
                patience=100,
                mode="max"
            )]
        )
        trainer.fit(model, loaders[0], loaders[1:])

        # dataframe store
        for idx, name in enumerate([
            "seen_mnist", "unseen_mnist", "unseen_fashion_mnist"
        ]):
            seq_acc, last_acc = seq_accuracy(loaders[idx+1])
            if n_tasks in n_tasks_for_full_check:
                prefix = f"experiments/{seq_res_output}_{n_tasks}"
                np.save(prefix+"_"+name, seq_acc)
            entry = {
                "n_tasks": n_tasks, "dataset": name,
                "last_acc": last_acc.item(),
            }
            logs.append(entry)
            pd.DataFrame(logs).to_csv(f"experiments/{output_csv}", index=0)
        clear_output(True)

Сначала запустить для MNIST, потом для FashionMNIST, вот и вся задача. Самое сложное - дождаться

In [ ]:
mnist_train = SubLoader(MNIST('./datasets', train=True, transform=g_transform)) 
train_for_sequence_improvement(mnist_train)

In [ ]:
mnist_train = SubLoader(FashionMnist('./datasets', train=True, transform=g_transform)) 
train_for_sequence_improvement(
    train_dataset=mnist_train,
    n_tasks_list=[2**16]
)

#### Интерпретация

Теперь будем строить графики по порядку

In [73]:
import pandas as pd

set_mapper = {
    "seen_mnist": "Seen MNIST",
    "unseen_mnist": "Unseen MNIST",
    "unseen_fashion_mnist": "Unseen FashionMNIST"
}
df = pd.read_csv("experiments/last_acc.csv")
df["dataset"] = df.dataset.map(set_mapper)

In [ ]:
import plotly.express as px

fig = px.line(
    df, "n_tasks", "last_acc", color="dataset",
    color_discrete_sequence=["crimson", "midnightblue", "goldenrod"]
)
fig.update_xaxes(
    type='category', tickvals=df.n_tasks.values,
    ticktext = [f"$2^{{{int(np.log2(i))}}}$" for i in df.n_tasks.values]
)
fig.update_layout(
    legend=dict(
        x=0.02,
        y=0.94,
        bgcolor="white",
        bordercolor="Black",
        borderwidth=2,
        title="Dataset"
    ),
    width=700, height=500, title="Ability to learn per number of tasks",
    yaxis=dict(title="Accuracy improvement in sequence"),
    xaxis=dict(title="Number of tasks"),
)
fig.add_vrect(
    x0=0, x1=5, 
    annotation_text="Instance memorization", annotation_position="top right",
    fillcolor="gold", opacity=0.05, line_width=0,
    label=dict(font=dict(size=12)),
)
fig.add_vrect(
    x0=5, x1=7, 
    annotation_text="Task memorization", annotation_position="top right",
    fillcolor="blue", opacity=0.05, line_width=0,
    label=dict(font=dict(size=12)),
)
fig.add_vrect(
    x0=7, x1=9, 
    annotation_text="Learning to learn", annotation_position="top left",
    fillcolor="red", opacity=0.05, line_width=0,
    label=dict(font=dict(size=12)),
)
fig

<img src="https://cdn.discordapp.com/attachments/674191702906503199/1194466609058811975/newplot_1.png?ex=65b074b7&is=659dffb7&hm=2006b8cee485c0a5ca054d4814c1356689931dc6605169f3ed2d468a9daf1f28&">

В целом график получается очень похожим, только качество слегка ниже, чем в статье. Но самое главное мы видим - модель действительно учится обощать при растущем количестве тасок

In [124]:
res = pd.concat([
    pd.DataFrame(np.load(f"experiments/seq_acc_{dataset}.npy"))
    for dataset in list(set_mapper)[1:]
])
res["dataset"] = [y for x in list(set_mapper)[1:] for y in [x]*99]

In [ ]:
fig = px.line(
    res, y=0, color="dataset",
    color_discrete_sequence=["midnightblue", "goldenrod"]
)
fig.update_layout(
    legend=dict(
        x=0.54,
        y=0.20,
        bgcolor="white",
        bordercolor="Black",
        borderwidth=2,
        title="Dataset"
    ),
    width=700, height=500, title="Ability to learn per number of tasks",
    yaxis=dict(title="Accuracy improvement in sequence"),
    xaxis=dict(title="Number of tasks"),
)
fig.update_xaxes(range=[-1, 101])
fig

<img src="https://cdn.discordapp.com/attachments/674191702906503199/1194466836012605481/newplot_2.png?ex=65b074ed&is=659dffed&hm=105d8d23f1bf527a4daa946d9c0155148c7932174caa594ab38a67546d254c2e&"> 